In [2]:
import boto3
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION_NAME = os.getenv('AWS_REGION_NAME')
AWS_S3_BUCKET_NAME = os.getenv('AWS_S3_BUCKET_NAME')

In [4]:
s3_client: boto3.client = boto3.client('s3',
                                           aws_access_key_id=AWS_ACCESS_KEY_ID,
                                           aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                                           region_name=AWS_REGION_NAME)

In [5]:
s3_client.list_objects(Bucket=AWS_S3_BUCKET_NAME)['Contents']

[{'Key': 'rates_2023_08_13.csv',
  'LastModified': datetime.datetime(2023, 8, 13, 16, 12, 22, tzinfo=tzutc()),
  'ETag': '"b592510806a961a9d25c7d99442e24a7"',
  'Size': 5819,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}}]